In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression

In [22]:
# Load Integrated Dataset (Member 2's Output)

df = pd.read_csv("02.integrated_telco_data.csv")
print("Loaded integrated data:", df.shape)
df.head()

Loaded integrated data: (55280, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,CUST00001,Male,0.0,No,Yes,3.0428,Yes,Yes,No,No,...,No,No,No,No,Month-to-Month,No,Mailed check,71.907980,159.305090,Yes
1,CUST00002,Male,1.0,Yes,No,3.0428,Yes,Yes,DSL,No,...,No,Yes,Unknown,No,One year,Yes,Bank transfer (automatic),21.351177,82.390090,No
2,CUST00003,Female,0.0,No,No,36.8700,Yes,Yes,DSL,No,...,No,Unknown,Yes,Yes,Month-to-Month,No,Electronic check,41.157794,1622.398519,Yes
3,CUST00005,Male,1.0,Yes,Yes,14.1083,Yes,Unknown,Fiber optic,Yes,...,Yes,No,No,No,Two year,Yes,Electronic check,21.351177,313.009691,Yes
4,CUST00006,Male,0.0,Yes,No,20.0413,Yes,No,Fiber optic,No,...,Unknown,Yes,Unknown,No,One year,No,Electronic check,34.567089,624.834330,Yes


Smoothing 

In [23]:
#smoothing

#remove noise in charges with regression

def smooth_charges(df, charge_col):
    valid = df[(df['tenure'].notna()) & (df['tenure'] > 0) & (df[charge_col].notna())]
    if len(valid) > 1:
        X = valid[['tenure']]
        y = valid[charge_col]
        reg = LinearRegression()
        reg.fit(X, y)
        y_pred = reg.predict(X)
        residuals = y - y_pred
        std_residuals = np.std(residuals)
        outliers = np.abs(residuals) > 3 * std_residuals
        if outliers.sum() > 0:
            df.loc[valid[outliers].index, charge_col] = reg.predict(valid[outliers][['tenure']])
            df[charge_col] = df[charge_col].clip(lower=0)  # Ensure non-negative
            print(f"Smoothed {outliers.sum()} outliers in {charge_col}")
    return df

df = smooth_charges(df, 'MonthlyCharges')


In [25]:
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,CUST00001,Male,0.0,No,Yes,3.0428,Yes,Yes,No,No,...,No,No,No,No,Month-to-Month,No,Mailed check,71.907980,159.305090,Yes
1,CUST00002,Male,1.0,Yes,No,3.0428,Yes,Yes,DSL,No,...,No,Yes,Unknown,No,One year,Yes,Bank transfer (automatic),21.351177,82.390090,No
2,CUST00003,Female,0.0,No,No,36.8700,Yes,Yes,DSL,No,...,No,Unknown,Yes,Yes,Month-to-Month,No,Electronic check,41.157794,1622.398519,Yes
3,CUST00005,Male,1.0,Yes,Yes,14.1083,Yes,Unknown,Fiber optic,Yes,...,Yes,No,No,No,Two year,Yes,Electronic check,21.351177,313.009691,Yes
4,CUST00006,Male,0.0,Yes,No,20.0413,Yes,No,Fiber optic,No,...,Unknown,Yes,Unknown,No,One year,No,Electronic check,34.567089,624.834330,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55275,CUST69995,Female,1.0,No,Yes,20.0413,Yes,Yes,Fiber optic,No,...,No,No,Yes,No,One year,Yes,Electronic check,41.157794,825.317601,No
55276,CUST69997,Male,0.0,Yes,Unknown,14.1083,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-Month,Yes,Bank transfer (automatic),41.157794,568.784330,Yes
55277,CUST69998,Male,0.0,Unknown,No,20.0413,Yes,No,DSL,Yes,...,No,No,Unknown,No,Month-to-Month,Yes,Mailed check,27.997485,556.898661,No
55278,CUST69999,Male,0.0,No,No,20.0413,Yes,No,Fiber optic,No,...,No,No,Yes,No,Month-to-Month,Yes,Electronic check,48.184182,1069.366824,No


In [14]:
df = smooth_charges(df, 'TotalCharges')

Smoothed 169 outliers in TotalCharges


Normalization

In [15]:

# Normalization

# Min-Max scaling for MonthlyCharges, TotalCharges
minmax = MinMaxScaler()
df[['MonthlyCharges','TotalCharges']] = minmax.fit_transform(df[['MonthlyCharges','TotalCharges']])

# Z-score scaling for tenure
zscore = StandardScaler()
df[['tenure']] = zscore.fit_transform(df[['tenure']])

df[['tenure','MonthlyCharges','TotalCharges']].head()


,tenure,MonthlyCharges,TotalCharges
0,-1.244740,1.000000,0.038073
1,-1.244740,0.000000,0.000000
2,1.104164,0.391770,0.762305
3,-0.476370,0.000000,0.114157
4,-0.064392,0.261407,0.268510


Feature Construction

In [16]:
#Feature Construction

# Average monthly charge
df['AvgMonthlyCharge'] = df['TotalCharges'] / (df['tenure'].fillna(0) + 1e-5)

# Count how many services a customer has
service_cols = ['PhoneService','MultipleLines','OnlineSecurity','OnlineBackup',
                'DeviceProtection','TechSupport','StreamingTV','StreamingMovies']

df['TotalServices'] = df[service_cols].apply(lambda row: sum(row=='Yes'), axis=1).fillna(0)

# Premium if more than 4 services
df['PremiumCustomer'] = (df['TotalServices'] > 4).astype(int)

# Indicator for electronic payment
electronic_payments = ['Electronic check','Bank transfer (automatic)','Credit card (automatic)']
df['ElectronicPayment'] = df['PaymentMethod'].isin(electronic_payments).astype(int)

df[['AvgMonthlyCharge','TotalServices','PremiumCustomer','ElectronicPayment']].head()


,AvgMonthlyCharge,TotalServices,PremiumCustomer,ElectronicPayment
0,-0.030587,2,0,0
1,-0.000000,3,0,1
2,0.690385,5,1,1
3,-0.239644,3,0,1
4,-4.170563,2,0,1


Discretization (Binning)

In [17]:
# Discretization

# Discretize tenure into Short, Medium, Long
df['TenureGroup'] = pd.qcut(df['tenure'], q=3, labels=['Short', 'Medium', 'Long'])

# Discretize MonthlyCharges, TotalCharges, AvgMonthlyCharge into Low, Medium, High
df['MonthlyChargeGroup'] = pd.qcut(df['MonthlyCharges'], 3, labels=['Low', 'Medium', 'High'])
df['TotalChargeGroup'] = pd.qcut(df['TotalCharges'], 3, labels=['Low', 'Medium', 'High'])
df['AvgMonthlyChargeGroup'] = pd.qcut(df['AvgMonthlyCharge'], 3, labels=['Low', 'Medium', 'High'])

df[['tenure','TenureGroup','MonthlyCharges','MonthlyChargeGroup']].head()


,tenure,TenureGroup,MonthlyCharges,MonthlyChargeGroup
0,-1.244740,Short,1.000000,High
1,-1.244740,Short,0.000000,Low
2,1.104164,Long,0.391770,Medium
3,-0.476370,Short,0.000000,Low
4,-0.064392,Medium,0.261407,Medium


Concept Hierarchy

In [18]:
#Concept Hierachy

# Map Contract and InternetService to ordinal levels

contract_hierarchy = {'Month-to-month': 1, 'One year': 2, 'Two year': 3}
df['ContractLevel'] = df['Contract'].map(contract_hierarchy).fillna(0)

internet_hierarchy = {'No': 1, 'DSL': 2, 'Fiber optic': 3}
df['InternetServiceLevel'] = df['InternetService'].map(internet_hierarchy).fillna(0)

df[['Contract','ContractLevel','InternetService','InternetServiceLevel']].head()


,Contract,ContractLevel,InternetService,InternetServiceLevel
0,Month-to-Month,NaN,No,1
1,One year,2.0,DSL,2
2,Month-to-Month,NaN,DSL,2
3,Two year,3.0,Fiber optic,3
4,One year,2.0,Fiber optic,3


Encoding Categorical Variables

In [19]:
# Encoding

# One-hot encoding for categorical
cat_cols = ['gender','InternetService','PaymentMethod','Contract',
            'TenureGroup','MonthlyChargeGroup','TotalChargeGroup','AvgMonthlyChargeGroup']

df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

# Convert Yes/No to 1/0
binary_cols = ['Partner', 'Dependents', 'PaperlessBilling', 'Churn']
for col in binary_cols:
    if col in df.columns:
        df[col] = df[col].map({'Yes': 1, 'No': 0}).fillna(0)

df.head()


,customerID,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,Contract_One year,Contract_Two year,TenureGroup_Medium,TenureGroup_Long,MonthlyChargeGroup_Medium,MonthlyChargeGroup_High,TotalChargeGroup_Medium,TotalChargeGroup_High,AvgMonthlyChargeGroup_Medium,AvgMonthlyChargeGroup_High
0,CUST00001,0.0,0.0,1.0,-1.244740,Yes,Yes,No,No,No,...,False,False,False,False,False,True,False,False,True,False
1,CUST00002,1.0,1.0,0.0,-1.244740,Yes,Yes,No,No,No,...,True,False,False,False,False,False,False,False,True,False
2,CUST00003,0.0,0.0,0.0,1.104164,Yes,Yes,No,Yes,No,...,False,False,False,True,True,False,False,True,False,True
3,CUST00005,1.0,1.0,1.0,-0.476370,Yes,Unknown,Yes,No,Yes,...,False,True,False,False,False,False,False,False,False,False
4,CUST00006,0.0,1.0,0.0,-0.064392,Yes,No,No,No,Unknown,...,True,False,True,False,True,False,True,False,False,False


In [20]:

df.to_csv("03.transformed_telco_data.csv", index=False)

print(" Data Transformation & Discretization complete. Saved as 03.transformed_telco_data.csv")
print("Final dataset shape:", df.shape)

 Data Transformation & Discretization complete. Saved as 03.transformed_telco_data.csv
Final dataset shape: (55280, 41)
